<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A8%D8%B3%D9%85_%D8%A7%D9%84%D9%84%D9%87_%D8%A7%D9%84%D8%B1%D8%AD%D9%85%D9%86_%D8%A7%D9%84%D8%B1%D8%AD%D9%8A%D9%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time  # استيراد مكتبة الوقت
import gc

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()


def free_memory():
    """تحرير الذاكرة غير المستخدمة."""
    gc.collect()  # استدعاء جمع القمامة لتحرير الذاكرة
    print("Memory has been freed.")


def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None


def calculate_atr(data, period=14):
    """Calculate Average True Range (ATR) for risk management."""
    high_low = data['high'] - data['low']
    high_close = abs(data['high'] - data['close'].shift(1))
    low_close = abs(data['low'] - data['close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr


def update_stop_loss(position, new_stop_loss):
    """تحديث وقف الخسارة للصفقة المحددة مع الحفاظ على تيك بروفت."""
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "position": position.ticket,
        "sl": new_stop_loss,
        "tp": position.tp,  # الحفاظ على تيك بروفت كما هو
        "symbol": position.symbol,
        "magic": position.magic,
        "deviation": 10,
        "comment": "Update Stop Loss",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    return result


def set_break_even_all_positions():
    """
    Sets the stop-loss to different levels based on price progress towards the target
    for all positions in the account, without moving the stop-loss if the new one is lower than the old one.
    """
    # Get all positions in the account
    all_positions = mt5.positions_get()

    if all_positions:
        for position in all_positions:
            symbol = position.symbol  # Get the symbol for the current position
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl

            # Determine the current price based on the order type
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # Calculate the distance to take profit (absolute value)
            distance_to_tp = abs(take_profit - entry_price)

            # Calculate targets (adjust for buy/sell)

            percent_target123 = entry_price + (0.10 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.10 * distance_to_tp)
            percent_target12 = entry_price + (0.20 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.20 * distance_to_tp)
            percent_target1 = entry_price + (0.30 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.30 * distance_to_tp)
            percent_target = entry_price + (0.40 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.40 * distance_to_tp)
            fifty_percent_target = entry_price + (0.50 * distance_to_tp) if position.type == 0 else entry_price - (
                        0.50 * distance_to_tp)
            fifty_target = entry_price + (0.60 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.60 * distance_to_tp)
            seventy_percent_target = entry_price + (
                    0.70 * distance_to_tp) if position.type == 0 else entry_price - (0.70 * distance_to_tp)

            seventy_five_percent_target = entry_price + (
                        0.80 * distance_to_tp) if position.type == 0 else entry_price - (0.80 * distance_to_tp)

            ninety_percent_target = entry_price + (0.90 * distance_to_tp) if position.type == 0 else entry_price - (
                        0.90 * distance_to_tp)

            new_stop_loss = current_stop_loss

            # Update stop-loss based on current price and order type

            if position.type == 0 and current_price >= percent_target123:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.60 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target123:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.60 * distance_to_tp))

            if position.type == 0 and current_price >= percent_target12:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.50 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target12:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.50 * distance_to_tp))


            if position.type == 0 and current_price >= percent_target1:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.40 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target1:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.40 * distance_to_tp))

            if position.type == 0 and current_price >= percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.03 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.03 * distance_to_tp))

            if position.type == 0 and current_price >= fifty_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.07 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.07 * distance_to_tp))

            if position.type == 0 and current_price >= fifty_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.12 * distance_to_tp))

            if position.type == 0 and current_price >= seventy_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.18 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.18 * distance_to_tp))

            if position.type == 0 and current_price >= seventy_five_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_five_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.24 * distance_to_tp))

            if position.type == 0 and current_price >= ninety_percent_target:  # Buy Order
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
            elif position.type == 1 and current_price <= ninety_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.48 * distance_to_tp))

            # Check if the new stop-loss is different and valid before updating
            if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                update_stop_loss(position, new_stop_loss)


# Call the function to set break-even for all positions
set_break_even_all_positions()


def calculate_lot_size(account_balance, risk_percentage=0.01, pip_value=10):
    """حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة."""
    risk_amount = account_balance * risk_percentage
    lot_size = risk_amount / pip_value  # يمكنك تعديل pip_value حسب الأداة المالية
    return max(0.01, lot_size)  # تأكد من أن حجم العقد لا يقل عن 0.01


def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data


def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        # تحقق من الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol=symbol)
        if open_positions:
            for position in open_positions:
                entry_price = position.price_open
                # تحقق من المسافة بين السعر الحالي وأي من الصفات المفتوحة
                if abs(price - entry_price) < 0.5 * atr_value:
                    print("السعر الحالي قريب جدًا من صفقة مفتوحة، لا يمكن فتح صفقة جديدة.")
                    return

        sl = price - 2 * atr_value if order_type == "buy" else price + 2 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(
                f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")


def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []


def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments


def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i - 1] and data['close'].iloc[i - 1] < data['close'].iloc[
                i - 2]:
                bullish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
            elif data['close'].iloc[i] < data['close'].iloc[i - 1] and data['close'].iloc[i - 1] > data['close'].iloc[
                i - 2]:
                bearish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks


def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount


def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i - 1]:
                supply_zones.append((data.index[i], data['high'].iloc[i - 1]))
            elif data['close'].iloc[i] < data['low'].iloc[i - 1]:
                demand_zones.append((data.index[i], data['low'].iloc[i - 1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones


def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None


def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }


def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        print(f"RSI: {data['RSI'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data


def calculate_stochastic(data, k_window=14, d_window=3):
    """حساب Stochastic Oscillator"""
    low_min = data['low'].rolling(window=k_window).min()
    high_max = data['high'].rolling(window=k_window).max()
    data['%K'] = 100 * (data['close'] - low_min) / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=d_window).mean()
    return data


def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = data['close'].rolling(window=period).mean()
        print(f"Moving Average (MA): {data['MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data


def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = data['close'].diff(periods=period)
        print(f"Momentum: {data['Momentum'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data


def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        if 'volume' in data.columns:
            data['Volume_MA'] = data['volume'].rolling(window=20).mean()
            print(f"Volume Moving Average: {data['Volume_MA'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data


def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data) - 4):
            if (data['close'].iloc[i] < data['close'].iloc[i - 1] and
                    data['close'].iloc[i + 1] < data['close'].iloc[i] and
                    data['close'].iloc[i + 2] > data['close'].iloc[i + 1] and
                    data['close'].iloc[i + 3] < data['close'].iloc[i + 2] and
                    data['close'].iloc[i + 4] > data['close'].iloc[i + 3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points


def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data


def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data


def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        atr = calculate_atr(data)
        data['ATR'] = atr
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 2)
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)
        print(
            f"ATR: {data['ATR'].iloc[-1]}, Stop Loss: {data['Stop_Loss'].iloc[-1]}, Take Profit: {data['Take_Profit'].iloc[-1]}")
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data


def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i - 1] and البيانات['close'].iloc[i - 1] > \
                    البيانات['close'].iloc[i - 2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i - 1], البيانات['close'].iloc[i - 1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i - 1] and البيانات['close'].iloc[i - 1] < \
                    البيانات['close'].iloc[i - 2]:
                مناطق_طلبات_البيع.append((البيانات.index[i - 1], البيانات['close'].iloc[i - 1]))

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")


def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))
        if average_sentiment > 0.1 and open_positions_count < 5:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 5:
            place_order("XAUUSD", "sell", volume=0.01)

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 5:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


def check_open_positions(symbol):
    """Check if there are any open positions for the given symbol."""
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0


# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    last_memory_cleanup = time.time()  # حفظ وقت آخر تنظيف للذاكرة
    while True:
        # جمع البيانات
        data = get_and_clean_gold_prices()

        if data is not None:
            # تحديث وقف الخسارة لجميع الصفقات
            set_break_even_all_positions()

            # بناء الاستراتيجية أو تنفيذ الأوامر
            build_strategy(data)

        # تحرير الذاكرة كل خمس ساعات
        current_time = time.time()
        if current_time - last_memory_cleanup >= 60 * 60:  # 5 ساعات
            free_memory()
            last_memory_cleanup = current_time  # تحديث وقت آخر تنظيف للذاكرة

        # تأخير زمني بين التكرارات
        time.sleep(5)  # يمكنك تعديل الوقت حسب الحاجة
